In [118]:
import pandas as pd
import numpy as np
import torch
from importlib import reload
from tqdm.notebook import tqdm
import pytorch_lightning as pl
from transformers import GPT2Config, GPT2Tokenizer

pl.seed_everything(42)

42

## config and tokenizer

In [114]:
import yaml

config = yaml.load(open('config.yaml'))
config

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


{'data': {'batch_size': 32,
  'max_len': 256,
  'csv_file': 'processed.csv',
  'tokenizer_name': 'microsoft/DialoGPT-small',
  'val_frac': 0.1},
 'model': {'n_positions': 256,
  'n_ctx': 256512,
  'n_embd': 768,
  'n_layer': 6,
  'n_head': 12,
  'n_inner': 1024},
 'opt': {'lr': 5e-05, 'max_epochs': 100, 'weight_decay': 0.001},
 'exp': {'save_dir': 'weights/',
  'gradient_accumulation_steps': 1,
  'max_grad_norm': 1.0}}

In [98]:
tokenizer = GPT2Tokenizer.from_pretrained(config['data']['tokenizer_name'])
tokenizer.pad_token = tokenizer.eos_token

In [115]:
config['model']['vocab_size'] = tokenizer.vocab_size
config['model']['pad_token_id'] = tokenizer.pad_token_id
config['model']['eos_token_id'] = tokenizer.eos_token_id
config['model']['bos_token_id'] = tokenizer.bos_token_id
config

{'data': {'batch_size': 32,
  'max_len': 256,
  'csv_file': 'processed.csv',
  'tokenizer_name': 'microsoft/DialoGPT-small',
  'val_frac': 0.1},
 'model': {'n_positions': 256,
  'n_ctx': 256512,
  'n_embd': 768,
  'n_layer': 6,
  'n_head': 12,
  'n_inner': 1024,
  'vocab_size': 50257,
  'pad_token_id': 50256,
  'eos_token_id': 50256,
  'bos_token_id': 50256},
 'opt': {'lr': 5e-05, 'max_epochs': 100, 'weight_decay': 0.001},
 'exp': {'save_dir': 'weights/',
  'gradient_accumulation_steps': 1,
  'max_grad_norm': 1.0}}

## data

In [100]:
df = pd.read_csv(config['data']['csv_file'])
df.head()

,from,text
0,soroush,هدیه ی اولیه تولد! :))
1,erf,Ey khafan tarin ❤❤❤❤ Zogh haye ziad Melted ...
2,soroush,😌😌😌❤️ wish U enjoy it! :))
3,erf,Kheili khube ha bia berim jaye chaartaar y per...
4,soroush,dar khedmatim! 😏 :)) albatte age shoma eftexa...


In [101]:
contexted = []
n = 7
for i in tqdm(range(n, len(df['text']))):
    row = []
    prev = i - 1 - n # we additionally substract 1, so row will contain current responce and 7 previous responces  
    for j in range(i, prev, -1):
        row.append(df['text'][j])
    contexted.append(row)  
    
columns = ['response', 'context'] 
columns = columns + ['context/'+str(i) for i in range(n-1)]
df = pd.DataFrame.from_records(contexted, columns=columns)
df.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,Hamash 😏😏😏,kudumesh??😏,Ziba bud 😏,dar khedmatim! 😏 :)) albatte age shoma eftexa...,Kheili khube ha bia berim jaye chaartaar y per...,😌😌😌❤️ wish U enjoy it! :)),Ey khafan tarin ❤❤❤❤ Zogh haye ziad Melted ...,هدیه ی اولیه تولد! :))
1,mokhlesim😏 mokh_less😏,Hamash 😏😏😏,kudumesh??😏,Ziba bud 😏,dar khedmatim! 😏 :)) albatte age shoma eftexa...,Kheili khube ha bia berim jaye chaartaar y per...,😌😌😌❤️ wish U enjoy it! :)),Ey khafan tarin ❤❤❤❤ Zogh haye ziad Melted ...
2,Ino midunam😏,mokhlesim😏 mokh_less😏,Hamash 😏😏😏,kudumesh??😏,Ziba bud 😏,dar khedmatim! 😏 :)) albatte age shoma eftexa...,Kheili khube ha bia berim jaye chaartaar y per...,😌😌😌❤️ wish U enjoy it! :))
3,u 2😏,Ino midunam😏,mokhlesim😏 mokh_less😏,Hamash 😏😏😏,kudumesh??😏,Ziba bud 😏,dar khedmatim! 😏 :)) albatte age shoma eftexa...,Kheili khube ha bia berim jaye chaartaar y per...
4,Haaamiiine ke has 😏,u 2😏,Ino midunam😏,mokhlesim😏 mokh_less😏,Hamash 😏😏😏,kudumesh??😏,Ziba bud 😏,dar khedmatim! 😏 :)) albatte age shoma eftexa...


In [109]:
import dataset
reload(dataset)
from dataset import get_dataloaders

train_loader, val_loader = get_dataloaders(tokenizer, df, 
                                           max_len=config['data']['max_len'], 
                                           batch_size=config['data']['batch_size'], 
                                           val_frac=config['data']['val_frac'])

train dataset has 93291 samples and val dataset has 10365 samples


## model

In [ ]:
import model
reload(model)
from model import ErfBot


model = ErfBot(config=GPT2Config(**config['model']), 
               **config['opt'])
model

In [ ]:
model.count_parameters()

## trainer

In [ ]:
logger = TensorBoardLogger(
    save_dir='logs/',
    name='gpt2_logs'
)

checkpoint = ModelCheckpoint(dirpath='weights/gpt2', 
                             filename='{epoch}-{val_loss:.2f}', 
                             monitor='val_loss',
                             save_top_k=1, 
                             period=1)

lr_logger = LearningRateMonitor(logging_interval='step')

## defining trainer
trainer = Trainer(benchmark=True, 
                  gpus=1, 
                  logger=logger, 
                  max_epochs=config['opt']['max_epochs'],
                  callbacks=[checkpoint, lr_logger])

In [ ]:
trainer.fit(model, train_loader, val_loader)